# 2.1 Document Clustering on Key Words

## 2.1.1 Data Import

In [2]:
import pandas as pd

# Load cleaned dataset in
metadata_dataset = pd.read_csv("../Processed_Dataset/cleaned_dataset.csv", keep_default_na=False, na_values=[""])
metadata_dataset

,Unnamed: 0,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views,transcript
0,0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...
1,1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre..."
2,2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...
4,4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2335,2407,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430,"So, Ma was trying to explain something to me a..."
2336,2408,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470,This is a picture of a sunset on Mars taken by...
2337,2409,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what

## 2.1.2 Key Word Extraction

In [3]:
import gensim
import nltk
import re

stop_list = nltk.corpus.stopwords.words("english")
stemmer = nltk.stem.porter.PorterStemmer()

# Converts text files into a corpus
corpus = nltk.corpus.PlaintextCorpusReader("./Transcripts/", ".+\.txt")

print(corpus)

<PlaintextCorpusReader in '/Users/circl/Desktop/Jarrett/School/Y4S2/Text Mining/Project/TED_Talks_Text_Mining_Project/2_Topic_Modelling/Transcripts'>


In [4]:
# Converts corpus into a list of documents
fids = corpus.fileids()
docs1 = [corpus.words(f) for f in fids]
docs2 = [[w.lower() for w in doc] for doc in docs1]
docs3 = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs2]
docs4 = [[w for w in doc if w not in stop_list] for doc in docs3]
docs5 = [[stemmer.stem(w) for w in doc] for doc in docs4]


In [5]:
dictionary = gensim.corpora.Dictionary(docs5)
print(dictionary)

# Convert the list of documents into sparse vectors
vecs1 = [dictionary.doc2bow(doc) for doc in docs5]
tfidf = gensim.models.TfidfModel(vecs1)

Dictionary(35153 unique tokens: ['abil', 'abstract', 'academ', 'accord', 'achiev']...)


In [ ]:
from gensim import similarities

# DOCUMENT RETRIEVAL????
similarity_index = similarities.SparseMatrixSimilarity(vecs1, len(dictionary))

test_vector = vecs1[100]
sims = similarity_index[test_vector]
sorted_sims = sorted(enumerate(sims), key = lambda item: -item[1])

print(list(enumerate(sorted_sims)))

## 2.1.3 Data Clustering

### Identifying Optimal K

In [ ]:
import numpy as np # linear algebra
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import string
import kmeans


import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline
plt.style.use('fivethirtyeight')


metadata_dataset = pd.read_csv("../Processed_Dataset/cleaned_dataset.csv", keep_default_na=False, na_values=[""])

data = metadata_dataset['transcript']

tfidf = TfidfVectorizer(
    max_features = 8000,
    stop_words = 'english',
     
)

tfidf.fit(data)
text = tfidf.transform(data)

print(text)

def find_optimal_clusters(data, max_k):
    iters = range(2, max_k+1, 2)
    
    sse = []
    for k in iters:
        sse.append(MiniBatchKMeans(n_clusters=k, init_size=1024, batch_size=2048, random_state=20).fit(data).inertia_)
        print('Fit {} clusters'.format(k))
        
    f, ax = plt.subplots(1, 1)
    ax.plot(iters, sse, marker='o')
    ax.set_xlabel('Cluster Centers')
    ax.set_xticks(iters)
    ax.set_xticklabels(iters)
    ax.set_ylabel('SSE')
    ax.set_title('SSE by Cluster Center Plot')
    
find_optimal_clusters(text, 150)

#vecs1 = [dictionary.doc2bow(doc) for doc in docs5]

#tf_idf = vecs1.fit_transform(data) 

  (0, 7980)	0.04264503776028476
  (0, 7974)	0.03322775219657659
  (0, 7973)	0.02473252429471134
  (0, 7972)	0.014814624029708505
  (0, 7968)	0.015124124737928487
  (0, 7967)	0.08677342669614738
  (0, 7961)	0.015344614674386123
  (0, 7952)	0.014144646572250898
  (0, 7949)	0.01637056626932489
  (0, 7948)	0.018116777477532806
  (0, 7938)	0.02128489760448723
  (0, 7935)	0.047848542491995584
  (0, 7925)	0.04532066745534612
  (0, 7922)	0.013761422709096732
  (0, 7915)	0.030346657633144383
  (0, 7914)	0.07574813178978497
  (0, 7911)	0.0390085548304415
  (0, 7910)	0.04167945816264829
  (0, 7909)	0.028440278201296097
  (0, 7888)	0.029338222914576287
  (0, 7869)	0.01788496871504227
  (0, 7848)	0.017390759601657896
  (0, 7847)	0.06649235846256395
  (0, 7834)	0.014241893335332084
  (0, 7819)	0.022245595640268487
  :	:
  (2339, 1242)	0.02904647800524741
  (2339, 1220)	0.03674099942911934
  (2339, 1218)	0.04935595221788516
  (2339, 1102)	0.025789193140250962
  (2339, 1050)	0.021263957326603834
  (23

In [16]:
print(text)

  (0, 7980)	0.04264503776028476
  (0, 7974)	0.03322775219657659
  (0, 7973)	0.02473252429471134
  (0, 7972)	0.014814624029708505
  (0, 7968)	0.015124124737928487
  (0, 7967)	0.08677342669614738
  (0, 7961)	0.015344614674386123
  (0, 7952)	0.014144646572250898
  (0, 7949)	0.01637056626932489
  (0, 7948)	0.018116777477532806
  (0, 7938)	0.02128489760448723
  (0, 7935)	0.047848542491995584
  (0, 7925)	0.04532066745534612
  (0, 7922)	0.013761422709096732
  (0, 7915)	0.030346657633144383
  (0, 7914)	0.07574813178978497
  (0, 7911)	0.0390085548304415
  (0, 7910)	0.04167945816264829
  (0, 7909)	0.028440278201296097
  (0, 7888)	0.029338222914576287
  (0, 7869)	0.01788496871504227
  (0, 7848)	0.017390759601657896
  (0, 7847)	0.06649235846256395
  (0, 7834)	0.014241893335332084
  (0, 7819)	0.022245595640268487
  :	:
  (2339, 1242)	0.02904647800524741
  (2339, 1220)	0.03674099942911934
  (2339, 1218)	0.04935595221788516
  (2339, 1102)	0.025789193140250962
  (2339, 1050)	0.021263957326603834
  (23